# 통신사 고객 분석을 통한 고객 이탈 예측

### 1. 데이터 전처리

In [17]:
import pandas as pd
import numpy as np

data = pd.read_csv('../../data/Telco_customer_churn_cleaned.csv')
data.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Streaming Music,Premium Tech Support,Unlimited Data,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Category
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Yes,0.0,0,20.94,129.09,1,Churned,Competitor
1,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Yes,0.0,0,18.24,169.89,2,Churned,Other


In [18]:
# No internet service -> No로 통일
data.loc[data['InternetService']=='No', ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies']] = 'No'

# DSL, Fiber optic -> Yes로 통일
data['InternetService'] = data['InternetService'].replace(['DSL', 'Fiber optic'], 'Yes') 

# No phone service -> No로 통일
data.loc[data['PhoneService']=='No', 'MultipleLines'] = 'No'

In [19]:
# Yes, No 문자열 데이터 -> 1과 0로 수정
target_cols = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
               'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn', 
                'Under 30', 'Married', 'Referred a Friend','Streaming Music', 
                'Premium Tech Support', 'Unlimited Data', 'InternetService']

for idx, row in data.iterrows():
    for col in target_cols:
        if (row[col] == 'Yes'):
            data.loc[idx, col] = 1
        elif (row[col] == 'No'):
            data.loc[idx, col] = 0

data[target_cols] = data[target_cols].astype('int64')

In [20]:
# Month-to-month:1, One year:12, Two year:24 로 변경
data.loc[data["Contract"]=='Month-to-month', "Contract"] = 1
data.loc[data["Contract"]=='One year', "Contract"] = 12
data.loc[data["Contract"]=='Two year', "Contract"] = 24

data["Contract"] = data["Contract"].astype('int64')

In [21]:
# Customer Status에 'stayed'와 'churned' 데이터만 남기기
data = data[data['Customer Status']!='Joined']

In [22]:
# 불필요한 컬럼 삭제
data.drop(columns=['customerID', 'Partner', 'churn_rate', 'Country', 'State', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude'], inplace=True)


In [23]:
# 컬럼 순서 정렬
data = data[[
    'gender', 'Age', 'Under 30', 'SeniorCitizen', 'Dependents', 'Married',
    'Contract', 'tenure', 'Referred a Friend', 'Number of Referrals', 'Offer',
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
    'StreamingTV', 'StreamingMovies', 'Streaming Music', 'Premium Tech Support', 'Unlimited Data', 
    'Avg Monthly GB Download', 'Avg Monthly Long Distance Charges', 'Total Extra Data Charges', 'Total Long Distance Charges', 
    'MonthlyCharges', 'TotalCharges', 'PaymentMethod', 'PaperlessBilling', 'Total Refunds', 'Total Revenue', 
    'Customer Status', 'Churn Reason', 'Churn Category', 'Satisfaction Score', 'CLTV', 'Churn', 'Count'
]]

In [24]:
# 컬럼명 수정
data.columns = [
    'gender', 'age', 'under30', 'senior', 'dependents', 'married',
    'contract', 'tenure', 'referred_friend', 'referrals_num', 'offer',
    'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 
    'streaming_tv', 'streaming_movies', 'streaming_music', 'premium_tech_support', 'unlimited_data', 
    'avg_monthly_GBdownload', 'avg_monthly_longDistance_charges', 'total_extraData_charges', 'total_longDistance_charges', 
    'monthly_charges', 'total_charges', 'payment_method', 'paperless_billing', 'total_refunds', 'total_revenue', 
    'customer_status', 'churn_reason', 'churn_category', 'satisfaction_score', 'CLTV', 'churn', 'count'
]

In [25]:
data.head(2)

,gender,age,under30,senior,dependents,married,contract,tenure,referred_friend,referrals_num,...,paperless_billing,total_refunds,total_revenue,customer_status,churn_reason,churn_category,satisfaction_score,CLTV,churn,count
0,Male,37,0,0,0,0,1,2,0,0,...,1,0.0,129.09,Churned,Competitor made better offer,Competitor,1,3239,1,1
1,Female,19,1,0,0,0,1,2,0,0,...,1,0.0,169.89,Churned,Moved,Other,2,2701,1,1


In [26]:
len(data.columns), data.columns

(40,
 Index(['gender', 'age', 'under30', 'senior', 'dependents', 'married',
        'contract', 'tenure', 'referred_friend', 'referrals_num', 'offer',
        'phone_service', 'multiple_lines', 'internet_service',
        'online_security', 'online_backup', 'device_protection', 'tech_support',
        'streaming_tv', 'streaming_movies', 'streaming_music',
        'premium_tech_support', 'unlimited_data', 'avg_monthly_GBdownload',
        'avg_monthly_longDistance_charges', 'total_extraData_charges',
        'total_longDistance_charges', 'monthly_charges', 'total_charges',
        'payment_method', 'paperless_billing', 'total_refunds', 'total_revenue',
        'customer_status', 'churn_reason', 'churn_category',
        'satisfaction_score', 'CLTV', 'churn', 'count'],
       dtype='object'))

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6589 entries, 0 to 7042
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gender                            6589 non-null   object 
 1   age                               6589 non-null   int64  
 2   under30                           6589 non-null   int64  
 3   senior                            6589 non-null   int64  
 4   dependents                        6589 non-null   int64  
 5   married                           6589 non-null   int64  
 6   contract                          6589 non-null   int64  
 7   tenure                            6589 non-null   int64  
 8   referred_friend                   6589 non-null   int64  
 9   referrals_num                     6589 non-null   int64  
 10  offer                             6589 non-null   object 
 11  phone_service                     6589 non-null   int64  
 12  multip